In [35]:
import random, numpy as np, scipy, math, sys, hashlib

In [36]:
def ow_hash(password):
    h = str(int(hashlib.sha256(str(password).encode('utf-8')).hexdigest(), 16) % 10**20)
    while (len(h)<L_MAX):
        h = '0' + h #leftside padding should maintain uniformity
    return h

#### Only the last line of code changes in the class Client.

In [37]:
class Client:
    def __init__(self,identity):
        self.ID = identity
        self.x = [] #will store the hashes
    def initiate_setup(self,server):
        p_A = input('Please enter a natural number of at most ' + str(L_MAX) + ' digits :')

        while((not isinstance(p_A, int) and  not isinstance(p_A, long)) or p_A >= 10**(L_MAX) or p_A < 0):
            p_A = input('Please enter a natural number of at most ' + str(L_MAX) + ' digits :')

        print('Input password :')
        print(p_A)
        #----------------------------------padding the password with zeros
        while (p_A < 10**(L_MAX-1) and p_A > 0):
            p_A = 10* p_A
        if (p_A == 0): p_A = '0'*(L_MAX) 
        p_A = str(p_A)
        print('Padded password :')
        print(p_A)
        #---------------------------hashing the padded password N times (and storing all values for simplicity)
        self.x = [] #reset the hashes
        self.x.append(ow_hash(p_A))
        for i in range(N-1):
            self.x.append(ow_hash(self.x[i]))
        server.setup(self.x[N-1],self.ID)
    
    def  request_challenge(self,server, identity):
            self.n, self.r_n = server.provide_challenge(identity) #send its identity and receive the challenge
    def  answer_challenge(self,server):   
            if (self.n > 0): #no errors occured
                server.verify(self.x[N-1-self.n], self.r_n) #sends back the hash corresponding to n and the received r_n

#### In the class server only the necessary number of cycles in the verify method changes.

In [38]:
class Server:
    def __init__(self):
        self.n =0 #will count the number of attempted authentications
        self.ID_reg = -1
    
    def setup(self,N_fold_hash,identity):
        if (self.ID_reg == -1 or self.ID_reg == identity):
            self.ID_reg = identity
            self.x_N = N_fold_hash
            self.c=0 # a new OTP entity auth. has started so n=0
            print 'N-Fold hashed password:', self.x_N , 'has been associated to identity:', self.ID_reg, '.'
        else :
            print('Setup faillure.')
    
    def provide_challenge(self, identity):
        print('Identity authentication request received.')
        if (self.ID_reg == identity and self.n < N):
            self.n = self.n + 1
            self.r_n = random.randrange(0,R)
            print('The challenge is [n, r_n] = [' + str(self.n) + ' , ' + str(self.r_n) + ']')
            return self.n , self.r_n
        else :
            if not self.ID_reg == identity : print('Identity not recognized')
            else : 
                if  not self.n < N : print('Max number of authentications reached for this password, please reinitiate setup.')
            return -1 , -1
    def verify(self,x,r):
        if(r == self.r_n):
            for i in range(self.n):
                x=ow_hash(x)
            if (x == self.x_N): print('Successful authentication as ' + str(self.ID_reg))
            else : print('Unsuccessful authentication.')
        else : print('Unsuccessful authentication.')

### Simulation of the protocol

In [39]:
#-----------------------------------------setting parameters
L_MAX = 20 # 20 digits are nearly equal to 64 bit
N= 1000 # not too large for n-fold hash computation time, but enough for a good number of runs before resetting the password
R= 2**256 #maximum range from which the challenge random nonce is extracted
#-----------------------------------------initialisation
A = Client('Pippo')
B = Server()
#-----------------------------------------simulation
A.initiate_setup(B)
for i in range(11):
    A.request_challenge(B,A.ID)
    A.answer_challenge(B)

Please enter a natural number of at most 20 digits :7
Input password :
7
Padded password :
70000000000000000000
N-Fold hashed password: 80999044949861642558 has been associated to identity: Pippo .
Identity authentication request received.
The challenge is [n, r_n] = [1 , 36749646599690202028284055258287136636726747769903053744162256369108287797988]
Successful authentication as Pippo
Identity authentication request received.
The challenge is [n, r_n] = [2 , 115557243371313536887934046761567590436552261865866163509464589518890707575025]
Successful authentication as Pippo
Identity authentication request received.
The challenge is [n, r_n] = [3 , 22476425828625871576657154553444558718085992978505789733920075453197945679081]
Successful authentication as Pippo
Identity authentication request received.
The challenge is [n, r_n] = [4 , 114351487120056181233199730225514563035498831006213514830182607897333786431401]
Successful authentication as Pippo
Identity authentication request received.
Th